In [1]:
import numpy as np

import seaborn as sns
import pandas as pd
import os
import pprint
import ast
import re
pp = pprint.PrettyPrinter(depth=6)

os.chdir(os.path.expanduser('~/vivarium-ecoli'))

import matplotlib.pyplot as plt
import dill
import requests
import xmltodict

In [2]:
complexation_rxn_df = pd.read_csv('reconstruction/ecoli/flat/complexation_reactions.tsv', sep='\t')
stoich_series = complexation_rxn_df.loc[:,['id', 'stoichiometry']].dropna().reset_index(drop=True)

stoich_list = []
for i, stoich in enumerate(stoich_series.loc[:, 'stoichiometry']):

    stoich = stoich.replace('null', '-1')

    stoich = ast.literal_eval(stoich)
    for k,v in stoich.items():
        stoich_list.append([k, v, stoich_series.loc[i, 'id'], None, None, None])

In [4]:
pre_complex_df = pd.DataFrame(stoich_list, columns=['complex', 'stoichiometry', 'reaction', 'enzyme-reaction', 'cofactors', 'metal_binding'])
pre_complex_df

,complex,stoichiometry,reaction,enzyme-reaction,cofactors,metal_binding
0,1-PFK,1,1-PFK_RXN,None,None,None
1,1-PFK-MONOMER,-2,1-PFK_RXN,None,None,None
2,2OXOGLUTARATEDEH-CPLX,1,2OXOGLUTARATEDEH-CPLX_RXN,None,None,None
3,E1O,-1,2OXOGLUTARATEDEH-CPLX_RXN,None,None,None
4,E2O,-1,2OXOGLUTARATEDEH-CPLX_RXN,None,None,None
...,...,...,...,...,...,...
2811,FFS-RNA,-1,SRP-CPLX_RXN,None,None,None
2812,CPLX0-7796APO,1,CPLX0-7796APO_RXN,None,None,None
2813,PD04032,-2,CPLX0-7796APO_RXN,None,None,None
2814,ARCB-CPLX,1,ARCB-CPLX_RXN,None,None,None


In [ ]:
s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'cellulararchitect@protonmail.com', 'password':'Cellman0451'})

In [12]:
# for each row of pre_complex_df, get the monomers of the complex and add to set of monomers.
for i in len(pre_complex_df.index):
    complex = pre_complex_df.loc[i, 'complex']
    req_str = f'https://websvc.biocyc.org/apixml?fn=monomers-of-protein&id=ECOLI:{complex}&detail=low'

    r = s.get(req_str)
    o = xmltodict.parse(r.content)



TypeError: 'int' object is not iterable

# Get the monomers of a complex

In [5]:
s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'cellulararchitect@protonmail.com', 'password':'Cellman0451'})

<Response [200]>

In [10]:
complex = 'LEUC-MONOMER'

req_str = f'https://websvc.biocyc.org/getxml?id=ECOLI:{complex}&detail=high'

r = s.get(req_str)

o = xmltodict.parse(r.content)['ptools-xml']['Protein']
pp.pprint(o)

{'@ID': 'ECOLI:LEUC-MONOMER',
 '@detail': 'full',
 '@frameid': 'LEUC-MONOMER',
 '@orgid': 'ECOLI',
 'citation': [{'Publication': {'@ID': 'ECOLI:PUB-320178',
                               '@detail': 'full',
                               '@frameid': 'PUB-320178',
                               '@orgid': 'ECOLI',
                               'author': [{'#text': 'Davis MG',
                                           '@datatype': 'string'},
                                          {'#text': 'Calvo JM',
                                           '@datatype': 'string'}],
                               'pubmed-id': {'#text': '320178',
                                             '@datatype': 'string'},
                               'source': {'#text': 'J Bacteriol 129(2);1078-90',
                                          '@datatype': 'string'},
                               'title': {'#text': 'Isolation and '
                                                  'characterization of lambd

In [11]:
req_str = 'https://websvc.biocyc.org/apixml?fn=monomers-of-protein&id=ECOLI:LEUC-MONOMER&detail=low'

r = s.get(req_str)

o = xmltodict.parse(r.content)
pp.pprint(o)

{'ptools-xml': {'@ptools-version': '27.0',
                '@xml:base': 'http://BioCyc.org/apixml?fn=monomers-of-protein%26id=ECOLI:LEUC-MONOMER%26detail=LOW',
                'Protein': {'@ID': 'ECOLI:LEUC-MONOMER',
                            '@detail': 'low',
                            '@frameid': 'LEUC-MONOMER',
                            '@orgid': 'ECOLI',
                            'common-name': {'#text': '3-isopropylmalate '
                                                     'dehydratase subunit LeuC',
                                            '@datatype': 'string'},
                            'component-of': {'Protein': {'@frameid': '3-ISOPROPYLMALISOM-CPLX',
                                                         '@orgid': 'ECOLI',
                                                         '@resource': 'getxml?ECOLI:3-ISOPROPYLMALISOM-CPLX'}},
                            'gene': {'Gene': {'@frameid': 'EG11576',
                                              '@orgid': 'E